
<p><img src="https://oceanprotocol.com/static/media/banner-ocean-03@2x.b7272597.png" alt="drawing" width="800" align="center"/>

<h1><center>Ocean Protocol - Manta Ray project</center></h1>
<h3><center>Decentralized Data Science and Engineering, powered by Ocean Protocol</center></h3>
<p>Version 0.5 - alpha</p>
<p>Package compatibility: squid-py v0.5.4, keeper-contracts 0.8.6, utilities 0.2.0,
<p>Component compatibility: Brizo v0.2.9, Aquarius v0.1.11, Nile testnet smart contracts 0.8.6</p>
<p><a href="https://github.com/oceanprotocol/mantaray">mantaray on Github</a></p>
<p>

# Getting Underway - Listing and searching registered assets
In this notebook, we will explore the concept of Assets. An Asset has an ID
(actually a 'decentralized' ID, called **DID**!).

An Asset also has a document which describes the Asset and how to # authorize and gain access (i.e. purchase the asset).
This document is called a **DDO**, the DID Document. For Data Scientists, the DDO attribute of note is the 'metadata'
attribute. Metadata is used to describe your asset, for example the name and description of a Data Set.

The DID is stored in the blockchain. The DDO is stored in a public searchable database, wrapped by the Aquarius
component.

*Note to the reader: This is a testnet: a simulated blockchain and simulated DDO store. This testnet is open to the
public, and therefore may change state in unexpected ways (your asset might get deleted, etc.)*

Further reading!

[W3C early draft standard 'Decentralized Identifiers (DIDs)'](https://w3c-ccg.github.io/did-spec/)

[OEP 7 - Ocean Protocol standard for 'Decentralized Identifiers'](https://github.com/oceanprotocol/OEPs/tree/master/7)

[OEP 7 - Ocean Protocol standard for 'Assets Metadata Ontology'](https://github.com/oceanprotocol/OEPs/tree/master/8)


### Section 0: Import modules, and setup logging

In [1]:
# Standard imports
import logging

# Import mantaray and the Ocean API (squid)
import squid_py
from squid_py.ocean.ocean import Ocean
from squid_py.config import Config
import mantaray_utilities as manta_utils

# Setup logging
manta_utils.logging.logger.setLevel('INFO')
print("squid-py Ocean API version:", squid_py.__version__)

10 - logging         - <module>        - Logging started


Using default logging settings.
squid-py Ocean API version: 0.5.11


In [2]:
# Get the configuration file path for this environment
CONFIG_INI_PATH = manta_utils.config.get_config_file_path()
logging.critical("Deployment type: {}".format(manta_utils.config.get_deployment_type()))
logging.critical("Configuration file selected: {}".format(CONFIG_INI_PATH))
logging.critical("Squid API version: {}".format(squid_py.__version__))

50 - <ipython-input-2-84136f362abd> - <module>        - Deployment type: USE_K8S_CLUSTER
50 - <ipython-input-2-84136f362abd> - <module>        - Configuration file selected: /home/batman/ocn/mantaray_jupyter/config_k8s_deployed.ini
50 - <ipython-input-2-84136f362abd> - <module>        - Squid API version: 0.5.11


In [3]:
# Instantiate Ocean from configuration file
configuration = Config(CONFIG_INI_PATH)
ocn = Ocean(configuration)

20 - ocean_secret_store - __init__        - 	SecretStore: url https://secret-store.dev-ocean.com, parity-client https://nile.dev-ocean.com, account 0x44E07D8190Cc0f0eCCC9C935D5EA47194301Cb00
20 - diagnostics     - verify_contracts - Keeper contract artifacts (JSON abi files) at: /home/batman/ocn/mantaray_jupyter/artifacts_nile
20 - diagnostics     - verify_contracts - Using keeper contracts from network nile, network id is 8995
20 - diagnostics     - verify_contracts - Looking for keeper contracts ending with ".nile.json", e.g. "AgreementStoreManager.nile.json".
20 - diagnostics     - verify_contracts - Finished loading keeper contracts:
	Dispenser: 0x865396b7ddc58C693db7FCAD1168E3BD95Fe3368
	OceanToken: 0x9861Da395d7da984D5E8C712c2EDE44b41F777Ad
	DIDRegistry: 0x4A0f7F763B1A7937aED21D63b2A78adc89c5Db23
	AgreementStoreManager: 0x62f84700b1A0ea6Bfb505aDC3c0286B7944D247C
	TemplateStoreManager: 0x9768c8ae44f1dc81cAA98F48792aA5730cAd2F73
	ConditionStoreManager: 0x39b0AA775496C5ebf26f3B81C9e

### Section 1: Assets are stored in the Metadata store (Aquarius) as a DDO
Anyone can search assets in the public metadata stores. Anyone can start their own metadata instance for thier
own marketplace.

The Metadata store is a database wrapped with a REST API.
For more details of functionality, see the documentation and our Swagger API page.

In [4]:
print("Aquarius metadata service URL: {}".format(configuration.aquarius_url))
print("Aquarius metadata service Swagger API page (try it out!): {}/api/v1/docs/".format(configuration.aquarius_url))

Aquarius metadata service URL: https://nginx-aquarius.dev-ocean.com/
Aquarius metadata service Swagger API page (try it out!): https://nginx-aquarius.dev-ocean.com//api/v1/docs/


Similarly, the access control service is called Brizo, and will manage any access requests for an asset.

In [5]:
print("Brizo access service URL: {}".format(configuration.get('resources','brizo.url')))
print("Brizo access service Swagger API page (try it out!): {}/api/v1/docs/".format(configuration.get('resources','brizo.url')))
#TODO: Swagger page is still broken

Brizo access service URL: https://nginx-brizo.dev-ocean.com/
Brizo access service Swagger API page (try it out!): https://nginx-brizo.dev-ocean.com//api/v1/docs/


### Section 2: Listing registered asset metadata in Aquarius
All stored assets can be listed. This is typically not done in production, as the list would be too large.
First retrieve a list of all DID's (Decentralized IDentifiers) from Aquarius using the 'exists' tag.
This is an example of the low level MongoDB API.
TODO: Seperate this into utils library, generally a user would not want to list all assets, could be a large list!

In [6]:
# Use the Query function to get all existing assets
basic_query = {"service":{"$elemMatch":{"metadata": {"$exists" : True }}}}
all_ddos = ocn.assets.query(basic_query)
assert len(all_ddos), "There are no assets registered, go to s03_publish_and_register!"
print("There are {} assets registered in the metadata store.".format(len(all_ddos)))

20 - ocean_assets    - query           - Searching asset query: {'service': {'$elemMatch': {'metadata': {'$exists': True}}}}


There are 100 assets registered in the metadata store.


Aquarius is a document store, with the key being the DID, and the document being the DDO
(DID Document). The DDO describes the asset (metadata) and how to access it (Service Execution Agreement).
For more information on these topics, please visit the Ocean Protocol standards;

[OEP 7 - Decentralized Identifiers](https://github.com/oceanprotocol/OEPs/tree/master/7)

[OEP 7 - Decentralized Identifiers](https://github.com/oceanprotocol/OEPs/tree/master/8)

Let's select an asset DDO for inspection (Note, since the database is stateful, this can easily change/break,
so try with another index or register your own asset first!)

In [7]:
# Select a single asset DDO from the list
this_ddo = all_ddos[-1]
print("Selected asset DID: {}".format(this_ddo.did))
print("Asset name:", this_ddo.metadata['base']['name'])
print("Asset price: {} token".format(this_ddo.metadata['base']['price']))
print("Asset description: {} token".format(this_ddo.metadata['base']['description']))

Selected asset DID: did:op:f5b5cae6f83243c2bd284e8b6d776847e319ee4a60f740ca995175c9696904c0
Asset name: Ocean protocol white paper
Asset price: 23 token
Asset description: Introduce the main concepts and vision behind ocean protocol token


### Section 4: Searching the Ocean
Aquarius supports query search. A list of [DDO] is returned from a search call.

Currently, Aquarius is running MongoDB. For detailed query documentation, see the
[documentation](https://docs.mongodb.com/manual/reference/method/db.collection.find/)

The exposed query endpoint is a subset of the full MongoDB search capability. For the documentation on the
Current search implementation, see https://github.com/oceanprotocol/aquarius/blob/develop/docs/for_api_users/API.md

In [8]:
# Query syntax:
# {
#   "query": {"name_of_query":["parameters"]},
#   "sort": {"field":1},
#   "offset": 100,
#   "page": 0
# }

#### Select ALL assets
To get started, the following query will return all documents with a 'price' between 0 and 20.
The syntax for this query, is a range of integers for the registered price.

In [9]:
query = {"query":{"price":[0,20]}}
search_results = ocn.assets.query(query)
print("Found {} assets".format(len(search_results)))
print_match_idx = -1
for result in search_results:
    print("Selected asset: {}, price:{}, {}".format(result.metadata['base']['name'],result.metadata['base']['price'], result.did ))

20 - ocean_assets    - query           - Searching asset query: {'query': {'price': [0, 20]}}


Found 32 assets
Selected asset: TestAsset, price:10, did:op:ba1dd9c8e43d487491dd2e8b30751787aa6c02061a2d47bbbbbd2d1145e431db
Selected asset: TestAsset, price:10, did:op:4fe6cbaf04ef4273a5620a98c9555e73fc6e54913f15476fbd6003289b7b1bf7
Selected asset: TestAsset, price:10, did:op:aded045fef2a40fd99539f87607c1c236e9ebfc898ee4bdcb10bdae16dbc4982
Selected asset: TestAsset, price:10, did:op:1498671d8fa04bf7a9f3a28e85ff84b807b376ce503440cbad9fe40aa7b97040
Selected asset: TestAsset, price:10, did:op:90b4e86a8fba4ba0b38daa297e085aebc45532b4e6bb4dea80b8f6d07cc5ac6e
Selected asset: TestAsset, price:10, did:op:8ccf7dc4df3b4fbbbb267c27068293c31867613b06b748ac96837d08dadba3bd
Selected asset: TestAsset, price:10, did:op:be319fa87c50411e9a8e4683779efd4197623195879d496baa1811ab339b2f08
Selected asset: Test14gfxqo3120j, price:10, did:op:d48e976b1e27408eadab4df5e14a60b0cec4d9a1b96e418eb324b7e19411234c
Selected asset: Test24gfxqo3120j, price:10, did:op:ca7e72fce6804f308aa445a185898b317efdb05b97134b25ac78d6

#### Text search
Plain text search is supported, searching in all assets

In [10]:
query = {"query":{"text":["Weather"]}}
search_results = ocn.assets.query(query)
print("Found {} assets".format(len(search_results)))
print_match_idx = -1
for result in search_results:
    print("Selected asset: {}, price:{}, {}".format(result.metadata['base']['name'],result.metadata['base']['price'], result.did ))

20 - ocean_assets    - query           - Searching asset query: {'query': {'text': ['Weather']}}


Found 66 assets
Selected asset: UK Weather information 2011, price:10, did:op:7c89aa22fed245a98d8129d86fd275638f677433420249e9b9d195c4479bbdc8
Selected asset: UK Weather information 2011, price:10, did:op:e873286909694f548c6a38238e0f7ae42bb01f5b4c9f4f19bdf5584a6f782cd3
Selected asset: UK Weather information 2011, price:10, did:op:109aff0e949a4af8b40d855e0cdeca0f5078978dd70a45f389f8c4992390393b
Selected asset: UK Weather information 2011, price:10, did:op:64e41d9049f24756919412e1f6e90bc8aada502d2263469ca3b5cb309d6080bc
Selected asset: UK Weather information 2011, price:10, did:op:e90a459b328444f7ad6ec15dc8e86c4fa8c48151117444538194c3f736fe24d6
Selected asset: TestAsset, price:10, did:op:ba1dd9c8e43d487491dd2e8b30751787aa6c02061a2d47bbbbbd2d1145e431db
Selected asset: TestAsset, price:10, did:op:4fe6cbaf04ef4273a5620a98c9555e73fc6e54913f15476fbd6003289b7b1bf7
Selected asset: TestAsset, price:10, did:op:aded045fef2a40fd99539f87607c1c236e9ebfc898ee4bdcb10bdae16dbc4982
Selected asset: UK Wea

#### Combined search
Multiple queries can be joined

In [11]:
query = {"query":{"text":["Weather"],"price":[0,11]}}
search_results = ocn.assets.query(query)
print("Found {} assets".format(len(search_results)))
print_match_idx = -1
for result in search_results:
    print("Selected asset: {}, price:{}, {}".format(result.metadata['base']['name'],result.metadata['base']['price'], result.did ))

20 - ocean_assets    - query           - Searching asset query: {'query': {'text': ['Weather'], 'price': [0, 11]}}


Found 26 assets
Selected asset: TestAsset, price:10, did:op:ba1dd9c8e43d487491dd2e8b30751787aa6c02061a2d47bbbbbd2d1145e431db
Selected asset: TestAsset, price:10, did:op:4fe6cbaf04ef4273a5620a98c9555e73fc6e54913f15476fbd6003289b7b1bf7
Selected asset: TestAsset, price:10, did:op:aded045fef2a40fd99539f87607c1c236e9ebfc898ee4bdcb10bdae16dbc4982
Selected asset: TestAsset, price:10, did:op:1498671d8fa04bf7a9f3a28e85ff84b807b376ce503440cbad9fe40aa7b97040
Selected asset: TestAsset, price:10, did:op:90b4e86a8fba4ba0b38daa297e085aebc45532b4e6bb4dea80b8f6d07cc5ac6e
Selected asset: TestAsset, price:10, did:op:8ccf7dc4df3b4fbbbb267c27068293c31867613b06b748ac96837d08dadba3bd
Selected asset: TestAsset, price:10, did:op:be319fa87c50411e9a8e4683779efd4197623195879d496baa1811ab339b2f08
Selected asset: Test14gfxqo3120j, price:10, did:op:d48e976b1e27408eadab4df5e14a60b0cec4d9a1b96e418eb324b7e19411234c
Selected asset: Test24gfxqo3120j, price:10, did:op:ca7e72fce6804f308aa445a185898b317efdb05b97134b25ac78d6

### Section 5: Cleaning the Ocean
A DID is registered on the blockchain, and can be resolved to a DID Document (DDO) as presented above.

Since the DDO exists on Aquarius and not in the blockchain, the DDO itself can be deleted. The DID trace can never be
deleted from the blockchain.

In [12]:
if 0:

    # Let's count how many ddo's are registered
    all_dids = ocn.assets._get_aquarius().list_assets()
    print("there are {} assets registered in the metadata store.".format(len(all_dids)))

    # let's delete the first ddo object.
    first_did = all_dids[0]
    print("selected ddo for deletion:", first_did)
    ocn.assets._get_aquarius().retire_asset_ddo(first_did)

    # again, let's count how many ddo's are registered
    all_dids = ocn.assets._get_aquarius().list_assets()
    print("there are now {} assets registered in the metadata store.".format(len(all_dids)))

In [ ]:
# Deleting all assets!
# Please don't delete all the assets, as other users may be testing the components!
if 0:
    all_dids = ocn.assets._get_aquarius().list_assets()
    for i, did in enumerate(all_dids):
        print("Deleting DDO {} - {}".format(i, did))
        ocn.assets._get_aquarius().retire_asset_ddo(did)